In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import xml
import xml.etree.ElementTree as ET
import cv2
import math
import warnings
from skimage.transform import resize
import os
from scipy import ndimage
import shutil

In [ ]:
# see if the noetbook accesses the GPU
tf.config.list_physical_devices("GPU")

In [ ]:
# Train data locations
block_0101 = '../../S_lab_TasselNet/Block_1_TN/Block_1_images_and_xml'
block_0102 = '../../S_lab_TasselNet/Block_2_TN/Block_2_images_and_xml'
block_0203 = '../../S_lab_TasselNet/Block_9_TN/Block_9_images_and_xml'
block_0301 = '../../S_lab_TasselNet/Block_13_TN/Block_13_images_and_xml'

train_blocks = [block_0101, block_0102, block_0203, block_0301]

# valid data location
block_0204 = '../../S_lab_TasselNet/Block_10_TN/Block_10_images_and_xml'

valid_blocks = [block_0204]

In [ ]:
# list all the image and the xml files for the train data
all_train_contents = []
for item in train_blocks:
    block_contents = os.listdir(item)
    block_contents.sort()
    print(len(block_contents))
    all_train_contents.append(block_contents)

In [ ]:
# need to separate the image and the xml files - define a function
def separate_img_xml(content_list):
    image_files = []
    xml_files = []
    for file in content_list:
        if file.split(".")[-1] == 'jpeg':
            image_files.append(file)
            image_files.sort()
        else:
            xml_files.append(file)
            xml_files.sort()
    return(image_files, xml_files)

In [ ]:
img_files_11, xml_files_11 = separate_img_xml(all_train_contents[0])
img_files_12, xml_files_12 = separate_img_xml(all_train_contents[1])
img_files_23, xml_files_23 = separate_img_xml(all_train_contents[2])
img_files_31, xml_files_31 = separate_img_xml(all_train_contents[3])

In [ ]:
print(len(img_files_11), len(xml_files_11))
print(len(img_files_12), len(xml_files_12))
print(len(img_files_23), len(xml_files_23))
print(len(img_files_31), len(xml_files_31))

In [ ]:
# separate the annotated and not annotated images
def separate_annot_and_not(img_list, xml_list):
    image_names = [name.split(".")[0] for name in xml_list]
    annotated_images = []
    not_annotated_images = []
    for file in img_list:
        if file.split('.')[0] in image_names:
            annotated_images.append(file)
        else:
            not_annotated_images.append(file)
    return(annotated_images, not_annotated_images)

In [ ]:
annot_img_11, not_annot_img_11 = separate_annot_and_not(img_files_11, xml_files_11)
annot_img_12, not_annot_img_12 = separate_annot_and_not(img_files_12, xml_files_12)
annot_img_23, not_annot_img_23 = separate_annot_and_not(img_files_23, xml_files_23)
annot_img_31, not_annot_img_31 = separate_annot_and_not(img_files_31, xml_files_31)

In [ ]:
# separete the vertically and horizontally annotated images
def separate_annnot_vertical_horizontal(annotated_images, all_xml_files, n):
    annot_vertical_images = annotated_images[:n]
    annot_vertical_xmls = all_xml_files[:n]
    annot_horizontal_images = annotated_images[n:]
    annot_horizontal_xmls = all_xml_files[n:]
    return(annot_vertical_images, annot_vertical_xmls, annot_horizontal_images, annot_horizontal_xmls)

In [ ]:
annot_vertical_images_11, annot_vertical_xmls_11, annot_horizontal_images_11, annot_horizontal_xmls_11 = separate_annnot_vertical_horizontal(annot_img_11, 
                                                                                                                                             xml_files_11, 6)
annot_vertical_images_12, annot_vertical_xmls_12, annot_horizontal_images_12, annot_horizontal_xmls_12 = separate_annnot_vertical_horizontal(annot_img_12, 
                                                                                                                                             xml_files_12, 6)
annot_vertical_images_23, annot_vertical_xmls_23, annot_horizontal_images_23, annot_horizontal_xmls_23 = separate_annnot_vertical_horizontal(annot_img_23,
                                                                                                                                             xml_files_23, 6)
annot_vertical_images_31, annot_vertical_xmls_31, annot_horizontal_images_31, annot_horizontal_xmls_31 = separate_annnot_vertical_horizontal(annot_img_31, 
                                                                                                                                             xml_files_31, 6)

In [ ]:
# check if the annotated horizontal and vertical file names match
def check_annot(annot_vertical_images, annot_vertical_xmls, annot_horizontal_images, annot_horizontal_xmls):
    # are the vertical image and xml files matching
    annot_vertical = []
    for i in range(len(annot_vertical_images)):
        if annot_vertical_images[i].split(".")[0] == annot_vertical_xmls[i].split(".")[0]:
            annot_vertical.append(1)
    print(len(annot_vertical), len(annot_vertical_images))

    annot_horizontal = []
    # are the horizontally annotated images and the xml files the same?
    for j in range(len(annot_horizontal_images)):
        if annot_horizontal_images[i].split(".")[0] == annot_horizontal_xmls[i].split(".")[0]:
            annot_horizontal.append(1)
    print(len(annot_horizontal), len(annot_horizontal_images))  

In [ ]:
print(check_annot(annot_vertical_images_11, annot_vertical_xmls_11, annot_horizontal_images_11, annot_horizontal_xmls_11))
print(check_annot(annot_vertical_images_12, annot_vertical_xmls_12, annot_horizontal_images_12, annot_horizontal_xmls_12))
print(check_annot(annot_vertical_images_23, annot_vertical_xmls_23, annot_horizontal_images_23, annot_horizontal_xmls_23))
print(check_annot(annot_vertical_images_31, annot_vertical_xmls_31, annot_horizontal_images_31, annot_horizontal_xmls_31))

In [ ]:
# now store the images as npy files? - Keep an eye out to see if this is really necessary

# define a function to store the images as npy files - need two functions for vertical and horizontal separately - beacuse all images are read as horizontal images else.
def store_images_as_np_arrays_vertical(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    read_image = ndimage.rotate(read_image, 270)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
annot_vertical_images_11

In [ ]:
store_path = 'Train_TN_data/vertical_annotated_images_as_np_arrays/'

In [ ]:
# train_blocks = [block_0101, block_0102, block_0203, block_0301]
# blk 11
im_shapes_annot_vertical_11 = []
for item in annot_vertical_images_11:
    img_shape = store_images_as_np_arrays_vertical(block_0101, item, store_path)
    im_shapes_annot_vertical_11.append(img_shape)
print("Block 11 done!")

# blk 12
im_shapes_annot_vertical_12 = []
for item in annot_vertical_images_12:
    img_shape = store_images_as_np_arrays_vertical(block_0102, item, store_path)
    im_shapes_annot_vertical_12.append(img_shape)
print("Block 12 done!")

# blk 23
im_shapes_annot_vertical_23 = []
for item in annot_vertical_images_23:
    img_shape = store_images_as_np_arrays_vertical(block_0203, item, store_path)
    im_shapes_annot_vertical_23.append(img_shape)
print("Block 23 done!")

# blk 31
im_shapes_annot_vertical_31 = []
for item in annot_vertical_images_31:
    img_shape = store_images_as_np_arrays_vertical(block_0301, item, store_path)
    im_shapes_annot_vertical_31.append(img_shape)
print("Block 31 done!")

In [ ]:
print(im_shapes_annot_vertical_11)
print(im_shapes_annot_vertical_12)
print(im_shapes_annot_vertical_23)
print(im_shapes_annot_vertical_31)

In [ ]:
# store the vertically not-annotated images
store_path = 'Train_TN_data/vertical_not_annotated_images_as_np_arrays/'

# blk 11
im_shapes_notannot_vertical_11 = []
for item in not_annot_img_11:
    img_shape = store_images_as_np_arrays_vertical(block_0101, item, store_path)
    im_shapes_notannot_vertical_11.append(img_shape)
print("Block 11 done!")

# blk 12
im_shapes_notannot_vertical_12 = []
for item in not_annot_img_12:
    img_shape = store_images_as_np_arrays_vertical(block_0102, item, store_path)
    im_shapes_notannot_vertical_12.append(img_shape)
print("Block 12 done!")

# blk 23
im_shapes_notannot_vertical_23 = []
for item in not_annot_img_23:
    img_shape = store_images_as_np_arrays_vertical(block_0203, item, store_path)
    im_shapes_notannot_vertical_23.append(img_shape)
print("Block 23 done!")

# blk 31 - the first image needs to be rotated here
im_shapes_notannot_vertical_31 = []
for item in not_annot_img_31[1:]:
    img_shape = store_images_as_np_arrays_vertical(block_0301, item, store_path)
    im_shapes_notannot_vertical_31.append(img_shape)
print("Block 31 done!")

In [ ]:
# save the outlier image in block 31 - the first vertical na image
def store_images_as_np_arrays_vertical_outlier(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    read_image = ndimage.rotate(read_image, 90)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
img_shape_na_0_31 = store_images_as_np_arrays_vertical_outlier(block_0301, not_annot_img_31[0], store_path)

In [ ]:
# define a function to store horizontal images
def store_images_as_np_arrays_horizontal(img_old_path, img_name, img_store_path):
    # join the path
    image_path = os.path.join(img_old_path, img_name)
    # read the image
    read_image = plt.imread(image_path)
    image_size = read_image.shape
    # show the image
    plt.imshow(read_image)
    plt.show()
    # save the image in new location
    np.save(img_store_path + '/' + img_name.split(".")[0] + '.npy', read_image)
    return(image_size)

In [ ]:
# store the horizontally annotated images
store_path = "Train_TN_data/horizontal_annotated_images_as_np_arrays"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_11 = []
for item in annot_horizontal_images_11:
    img_shape = store_images_as_np_arrays_horizontal(block_0101, item, store_path)
    im_shapes_annot_horizontal_11.append(img_shape)
print("Block 11 done!")

# blk 12
im_shapes_annot_horizontal_12 = []
for item in annot_horizontal_images_12:
    img_shape = store_images_as_np_arrays_horizontal(block_0102, item, store_path)
    im_shapes_annot_horizontal_12.append(img_shape)
print("Block 12 done!")

# blk 23
im_shapes_annot_horizontal_23 = []
for item in annot_horizontal_images_23:
    img_shape = store_images_as_np_arrays_horizontal(block_0203, item, store_path)
    im_shapes_annot_horizontal_23.append(img_shape)
print("Block 23 done!")

# blk 31
im_shapes_annot_horizontal_31 = []
for item in annot_horizontal_images_31:
    img_shape = store_images_as_np_arrays_horizontal(block_0301, item, store_path)
    im_shapes_annot_horizontal_31.append(img_shape)
print("Block 31 done!")

create density maps

In [ ]:
# maybe copy all the xml files corresponding to the train data into a single location? That would be easier with data prep I believe
# train_blocks
# train_blocks = [block_0101, block_0102, block_0203, block_0301]
new_store_path = 'Train_TN_data/train_xml_files'
# blk 11
for file in xml_files_11:
    joined_path = os.path.join(block_0101, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

# blk 12
for file in xml_files_12:
    joined_path = os.path.join(block_0102, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

# blk 23
for file in xml_files_23:
    joined_path = os.path.join(block_0203, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

# blk 31
for file in xml_files_31:
    joined_path = os.path.join(block_0301, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# for vertically annotated images
def get_density_maps_vertical(file_name, image_path, xml_path, save_density_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    warnings.filterwarnings("ignore")
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 768, 'mid_x'] = 767
    mid_coords.loc[mid_coords['mid_y'] > 1024, 'mid_y'] = 1023
    

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)

    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = np.load(imge_file_path)
    # read_image_again = ndimage.rotate(read_image_again, 270)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [ ]:
# get the outputs from this function

image_path = 'Train_TN_data/vertical_annotated_images_as_np_arrays/'
xml_path = 'Train_TN_data/train_xml_files'
save_density_path = image_path

In [ ]:
# os.listdir(image_path)

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va = []
shapes_all_va = []
tasselCounts_all_va = []
tasselDensities_all_va = []
dense_maps_all_va = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va.append(name_va)
    shapes_all_va.append(shape_va)
    tasselCounts_all_va.append(tassels_va)
    tasselDensities_all_va.append(tassel_dense_va)
    dense_maps_all_va.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va == np.round(tasselDensities_all_va))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot = pd.DataFrame(zip(names_all_va, tasselCounts_all_va), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot.shape

In [ ]:
True_tasselcounts_df_vertical_annot.head()

In [ ]:
# save the true counts
True_tasselcounts_df_vertical_annot.to_csv("Train_TN_data/True_tassel_counts/vertical_annotated_true_counts.csv", index = False)

In [ ]:
# create density maps for the horizontally annotated images
def get_density_maps_horizontal(file_name, image_path, xml_path, save_density_path):
    xml_file = file_name + '.xml'
    xml_file_path = os.path.join(xml_path, xml_file)

    # Get coords from the xml file
    # parse the xml file
    parsed_file = ET.parse(xml_file_path)
    # get the roots
    root = parsed_file.getroot()
    # get the roots here
    coords = []
    for child in root:
        for i in child:
            for j in i:
                coords.append(int(j.text))
    
    # chunk the points into sets of 4 - these are the coordinates of the bounding boxes
    points_tupples = []
    for i in range(0, len(coords), 4):
        points_tupples.append(coords[i:i + 4])

    # make a dataframe with these points
    coords_df = pd.DataFrame(points_tupples, columns = ["bleft_x", "bleft_y", "tright_x", "tright_y"])

    # compute the number of tassels in each image
    no_of_tassels = len(points_tupples)

    # compute the mid coordinates
    coords_df["mid_x"] = (round(0.5*(coords_df["bleft_x"] + coords_df["tright_x"]))).astype(int)
    coords_df["mid_y"] = (round(0.5*(coords_df["bleft_y"] + coords_df["tright_y"]))).astype(int)

    # extract the mid cordinates
    mid_coords = coords_df[["mid_x", "mid_y"]]
    # cap the coords at the max height and width values
    mid_coords.loc[mid_coords['mid_x'] > 1024, 'mid_x'] = 1023
    mid_coords.loc[mid_coords['mid_y'] > 768, 'mid_y'] = 767
    warnings.filterwarnings("ignore")

    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)
    # check the shape of the read image
    read_image_shape = read_image.shape
    #  plot the bounding boxes on the image
    for points in points_tupples:
        annotated_image = cv2.rectangle(read_image, (points[0],points[1]), (points[2],points[3]), color = (255,0,0), thickness = 2)
    # plt.figure(figsize = (12,18))
    plt.imshow(annotated_image)
    plt.show()

    # plot the mid points on the image
    coords_list = mid_coords.values.tolist()
    # read the image again
    read_image_again = np.load(imge_file_path)
    # draw the circles on image
    for i in coords_list:
        image_with_mids = cv2.circle(read_image_again, i, radius=5, color=(255, 0, 0), thickness=-1)
    # look at the annotated image
    # plt.figure(figsize = (12,18))
    plt.imshow(image_with_mids)
    plt.show()

    # also try creating the density map here
    # first create the empty maps
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))
    # get the dot maps
    for point in coords_list:
        np_image[point[1], point[0]] = 1
    # plot the image
    # plt.figure(figsize = (12,18))
    plt.imshow(np_image, cmap = "Greys")
    plt.show()

    # now define the kernel and run the convolution
    one_d_kerenel = cv2.getGaussianKernel(50,5)
    two_d_kernel = np.multiply(one_d_kerenel.T, one_d_kerenel)

    # Shape of the 2D kernel
    twoD_shape = two_d_kernel.shape
        
    # do the convolution
    convolution = ndimage.convolve(np_image, two_d_kernel)
        
    # plot the density map
    # plt.figure(figsize = (12,18))
    plt.imshow(convolution, cmap = "Greys")
    plt.show()
        
    # get the sums of the images
    img_sum = np.sum(convolution)

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', convolution)

    return(file_name, read_image_shape, no_of_tassels, img_sum, convolution)

In [ ]:
# get the outputs from this function
image_path_h = "Train_TN_data/horizontal_annotated_images_as_np_arrays"
xml_path_h = 'Train_TN_data/train_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
# file_names_h

In [ ]:
%%time
names_all_h = []
shapes_all_h = []
tasselCounts_all_h = []
tasselDensities_all_h = []
dense_maps_all_h = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h.append(name_h)
    shapes_all_h.append(shape_h)
    tasselCounts_all_h.append(tassels_h)
    tasselDensities_all_h.append(tassel_dense_h)
    dense_maps_all_h.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h == np.round(tasselDensities_all_h))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot = pd.DataFrame(zip(names_all_h, tasselCounts_all_h), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot.shape

In [ ]:
True_tasselcounts_df_horizontal_annot.head()

In [ ]:
# save the true counts
True_tasselcounts_df_horizontal_annot.to_csv("Train_TN_data/True_tassel_counts/horizontal_annotated_true_counts.csv", index = False)

In [ ]:
# for vertically annotated images
def get_density_maps_not_annotated(file_name, image_path, save_density_path):
    
    # plot the bounding boxes on images
    # get image name and path
    image_name = file_name + '.npy'
    imge_file_path = os.path.join(image_path, image_name)
    # read the image
    read_image = np.load(imge_file_path)

    plt.imshow(read_image)
    plt.show()

    read_image_shape = read_image.shape
    
    # also try creating the density map here
    np_image = np.zeros((read_image_shape[0], read_image_shape[1]))

    # save the density map
    np.save(save_density_path + '/' + file_name + '_density_map.npy', np_image)

    img_sum = np.sum(np_image)

    return(file_name, read_image_shape, img_sum, np_image)

In [ ]:
image_path_vn = 'Train_TN_data/vertical_not_annotated_images_as_np_arrays/'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn = []
shapes_all_vn = []
tasselDensities_all_vn = []
dense_maps_all_vn = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn.append(name_vn)
    shapes_all_vn.append(shape_vn)
    tasselDensities_all_vn.append(tassel_dense_vn)
    dense_maps_all_vn.append(dense_map_vn)

In [ ]:
Vertical_not_annotated_df = pd.DataFrame(zip(names_all_vn, tasselDensities_all_vn), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df.head()

In [ ]:
Vertical_not_annotated_df.shape

In [ ]:
# save the true counts
Vertical_not_annotated_df.to_csv("Train_TN_data/True_tassel_counts/vertical_not_annotated_true_counts.csv", index = False)

In [ ]:
# move all the image np and density xml files to a single location

va_images_dense = 'Train_TN_data/vertical_annotated_images_as_np_arrays/'
vn_images_dense = 'Train_TN_data/vertical_not_annotated_images_as_np_arrays/'
h_images_dense = 'Train_TN_data/horizontal_annotated_images_as_np_arrays/'

In [ ]:
%%time
shutil.copytree(va_images_dense, 'Train_TN_data/train_all_image_density_files/', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Train_TN_data/train_all_image_density_files/', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Train_TN_data/train_all_image_density_files/', dirs_exist_ok=True)

In [ ]:
# create sub windows and counts - but does not seem like these were stored

In [ ]:
def create_subwindows_and_counts(image, numpy_folder, stride = 8, kernel_size = 32):
    im_name = image.split(".")[0]
    im_file = im_name + '.npy'
    count_file = im_name + '_density_map.npy'
    # load the image and the count numpy files
    loaded_im_file = np.load(numpy_folder + '/' + im_file)
    loaded_count_file = np.load(numpy_folder + '/' + count_file)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_image = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = loaded_count_file[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)
            sub_window = resize(sub_window, (32, 32,3))
            catch_image.append(sub_window)

    # save the combined subwindows and counts
    return(catch_image,density_sums, im_file)

In [ ]:
# let's do this for a sample and then in the loop
train_files_path = "Train_TN_data/train_all_image_density_files/"

In [ ]:
train_im_and_map_contents = os.listdir(train_files_path)

# sort these - ALWAYS sort these as the order is always messed up on HCC
train_im_and_map_contents.sort()

In [ ]:
len(train_im_and_map_contents)

In [ ]:
# get only the names of the image (npy files)
train_im_names = [item for item in train_im_and_map_contents if item.split(".")[0][-3:] != 'map']

In [ ]:
# how many? should be 32
len(train_im_names)

In [ ]:
# train_im_names

In [ ]:
# %%time
# # create the subwindows for all train data
# catch_all_image_subwindows_train = []
# catch_all_dense_subwindows_train = []
# catch_train_names = []
# for image in train_im_names:
#     train_ims, train_maps, train_names = create_subwindows_and_counts(image, train_files_path, stride = 8, kernel_size = 32)
#     catch_all_image_subwindows_train.append(train_ims)
#     catch_all_dense_subwindows_train.append(train_maps)
#     catch_train_names.append(train_names)

In [ ]:
# # sanity check names
# np.mean(train_im_names == catch_train_names)

In [ ]:
# notice that stacking is not done here, but done separately

Prepare validation data

In [ ]:
# list all the image and the xml files for the train data
all_valid_contents = os.listdir(block_0204)

In [ ]:
# separate image and xml files
img_files_24, xml_files_24 = separate_img_xml(all_valid_contents)

In [ ]:
# xml_files_24

In [ ]:
# separate annotated and not annotated images
annot_img_24, not_annot_img_24 = separate_annot_and_not(img_files_24, xml_files_24)

In [ ]:
not_annot_img_24

In [ ]:
# separate vertical and horizontal annotated images
annot_vertical_images_24, annot_vertical_xmls_24, annot_horizontal_images_24, annot_horizontal_xmls_24 = separate_annnot_vertical_horizontal(annot_img_24, 
                                                                                                                                             xml_files_24, 8)

In [ ]:
# check shapes
print(check_annot(annot_vertical_images_24, annot_vertical_xmls_24, annot_horizontal_images_24, annot_horizontal_xmls_24))

In [ ]:
# store the vertically annotated images - validation data
store_path = "Train_TN_data/vertical_annotated_images_as_np_arrays_valid"

# blk 24
im_shapes_annot_vertical_24 = []
for item in annot_vertical_images_24:
    img_shape = store_images_as_np_arrays_vertical(block_0204, item, store_path)
    im_shapes_annot_vertical_24.append(img_shape)
print("Block 24 done!")

In [ ]:
print(im_shapes_annot_vertical_24)

In [ ]:
# store the vertically not-annotated images
store_path = "Train_TN_data/vertical_not_annotated_images_as_np_arrays_valid"

# blk 24
im_shapes_notannot_vertical_24 = []
for item in not_annot_img_24:
    img_shape = store_images_as_np_arrays_vertical(block_0204, item, store_path)
    im_shapes_notannot_vertical_24.append(img_shape)
print("Block 24 done!")

In [ ]:
print(im_shapes_notannot_vertical_24)

In [ ]:
# store the horizontally annotated images
store_path = "Train_TN_data/horizontal_annotated_images_as_np_arrays_valid"

# blk 11 - the first horizontal image has a problem, needs to be rotated
im_shapes_annot_horizontal_24 = []
for item in annot_horizontal_images_24:
    img_shape = store_images_as_np_arrays_horizontal(block_0204, item, store_path)
    im_shapes_annot_horizontal_24.append(img_shape)
print("Block 24 done!")

In [ ]:
# Get the xml file paths in a single location
new_store_path = 'Train_TN_data/valid_xml_files'
# blk 11
for file in xml_files_24:
    joined_path = os.path.join(block_0204, file)
    # store the file in new path
    shutil.copy(joined_path, new_store_path)

In [ ]:
# get the outputs from this function
image_path = "Train_TN_data/vertical_annotated_images_as_np_arrays_valid/"
xml_path = 'Train_TN_data/valid_xml_files'
save_density_path = image_path

In [ ]:
file_names_v = [file.split(".")[0] for file in os.listdir(image_path) if file.split(".")[0][-3:] != 'map']
file_names_v.sort()

In [ ]:
# file_names_v

In [ ]:
%%time
names_all_va_v = []
shapes_all_va_v = []
tasselCounts_all_va_v = []
tasselDensities_all_va_v = []
dense_maps_all_va_v = []
for file_name in file_names_v:
    name_va, shape_va, tassels_va, tassel_dense_va, dense_map_va = get_density_maps_vertical(file_name, image_path, xml_path, save_density_path)
    names_all_va_v.append(name_va)
    shapes_all_va_v.append(shape_va)
    tasselCounts_all_va_v.append(tassels_va)
    tasselDensities_all_va_v.append(tassel_dense_va)
    dense_maps_all_va_v.append(dense_map_va)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_va == np.round(tasselDensities_all_va))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_vertical_annot_valid = pd.DataFrame(zip(names_all_va_v, tasselCounts_all_va_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_vertical_annot_valid.head()

In [ ]:
True_tasselcounts_df_vertical_annot_valid.shape

In [ ]:
# save the true counts
True_tasselcounts_df_vertical_annot_valid.to_csv("Train_TN_data/True_tassel_counts/vertical_annotated_true_counts_valid.csv", index = False)

In [ ]:
# get the outputs from this function
image_path_h = "Train_TN_data/horizontal_annotated_images_as_np_arrays_valid/"
xml_path_h = 'Train_TN_data/valid_xml_files'
save_density_path_h = image_path_h

In [ ]:
file_names_h = [file.split(".")[0] for file in os.listdir(image_path_h) if file.split(".")[0][-3:] != 'map']
file_names_h.sort()

In [ ]:
%%time
names_all_h_v = []
shapes_all_h_v = []
tasselCounts_all_h_v = []
tasselDensities_all_h_v = []
dense_maps_all_h_v = []
for file_h in file_names_h:
    name_h, shape_h, tassels_h, tassel_dense_h, dense_map_h = get_density_maps_horizontal(file_h, image_path_h, xml_path_h, save_density_path_h)
    names_all_h_v.append(name_h)
    shapes_all_h_v.append(shape_h)
    tasselCounts_all_h_v.append(tassels_h)
    tasselDensities_all_h_v.append(tassel_dense_h)
    dense_maps_all_h_v.append(dense_map_h)

In [ ]:
# a few sanity checks
np.mean(tasselCounts_all_h == np.round(tasselDensities_all_h))

In [ ]:
# create a dataframe of the true counts
True_tasselcounts_df_horizontal_annot_valid = pd.DataFrame(zip(names_all_h_v, tasselCounts_all_h_v), columns = ['name', 'true_count'])

In [ ]:
True_tasselcounts_df_horizontal_annot_valid.shape

In [ ]:
True_tasselcounts_df_horizontal_annot_valid.head()

In [ ]:
# save the true counts
True_tasselcounts_df_horizontal_annot_valid.to_csv("Train_TN_data/True_tassel_counts/horizontal_annotated_true_counts_valid.csv", index = False)

In [ ]:
image_path_vn = 'Train_TN_data/vertical_not_annotated_images_as_np_arrays_valid/'
save_density_path_vn = image_path_vn

In [ ]:
file_names_vn = [file.split(".")[0] for file in os.listdir(image_path_vn) if file.split(".")[0][-3:] != 'map']
file_names_vn.sort()

In [ ]:
len(file_names_vn)

In [ ]:
%%time
# get the density maps for all vertically annotated image files
names_all_vn_v = []
shapes_all_vn_v = []
tasselDensities_all_vn_v = []
dense_maps_all_vn_v = []
for file_vn in file_names_vn:
    name_vn, shape_vn, tassel_dense_vn, dense_map_vn = get_density_maps_not_annotated(file_vn, image_path_vn, save_density_path_vn)
    names_all_vn_v.append(name_vn)
    shapes_all_vn_v.append(shape_vn)
    tasselDensities_all_vn_v.append(tassel_dense_vn)
    dense_maps_all_vn_v.append(dense_map_vn)

In [ ]:
Vertical_not_annotated_df_valid = pd.DataFrame(zip(names_all_vn_v, tasselDensities_all_vn_v), columns = ['name', 'true_count'])

In [ ]:
Vertical_not_annotated_df_valid.head()

In [ ]:
Vertical_not_annotated_df_valid.shape

In [ ]:
# save the true counts
Vertical_not_annotated_df.to_csv("Train_TN_data/True_tassel_counts/vertical_not_annotated_true_countsvalid.csv", index = False)

In [ ]:
# move all the image np and density xml files to a single location

va_images_dense = 'Train_TN_data/vertical_annotated_images_as_np_arrays_valid/'
vn_images_dense = 'Train_TN_data/vertical_not_annotated_images_as_np_arrays_valid/'
h_images_dense = 'Train_TN_data/horizontal_annotated_images_as_np_arrays_valid/'

In [ ]:
%%time
shutil.copytree(va_images_dense, 'Train_TN_data/valid_all_image_density_files/', dirs_exist_ok=True)
shutil.copytree(vn_images_dense, 'Train_TN_data/valid_all_image_density_files/', dirs_exist_ok=True)
shutil.copytree(h_images_dense, 'Train_TN_data/valid_all_image_density_files/', dirs_exist_ok=True)

In [ ]:
# let's do this for a sample and then in the loop
valid_files_path = "Train_TN_data/valid_all_image_density_files/"

In [ ]:
valid_im_and_map_contents = os.listdir(valid_files_path)

# sort these - ALWAYS sort these as the order is always messed up on HCC
valid_im_and_map_contents.sort()

In [ ]:
len(valid_im_and_map_contents)

In [ ]:
# get only the names of the image (npy files)
valid_im_names = [item for item in valid_im_and_map_contents if item.split(".")[0][-3:] != 'map']

In [ ]:
len(valid_im_names)

In [ ]:
# valid_im_names

In [ ]:
%%time
# create the subwindows for all train data
catch_all_image_subwindows_valid = []
catch_all_dense_subwindows_valid = []
catch_valid_names = []
for image in valid_im_names:
    valid_ims, valid_maps, valid_names = create_subwindows_and_counts(image, valid_files_path, stride = 8, kernel_size = 32)
    catch_all_image_subwindows_valid.append(valid_ims)
    catch_all_dense_subwindows_valid.append(valid_maps)
    catch_valid_names.append(valid_names)

In [ ]:
# sanity check names
np.mean(valid_im_names == catch_valid_names)

In [ ]:
# stack the images
valid_im_stack = np.vstack(catch_all_image_subwindows_valid)
print(valid_im_stack.shape)

# stack the subcounts
valid_count_stack = np.hstack(catch_all_dense_subwindows_valid)
print(valid_count_stack.shape)

In [ ]:
393216/32

In [ ]:
index = 31
for i in range(index):
    print(np.mean(valid_im_stack[12288*index:12288+12288*index,:,:,:] == catch_all_image_subwindows_valid[index]), np.mean(valid_count_stack[12288*index:12288+12288*index,] == catch_all_dense_subwindows_valid[index]))

In [ ]:
# do a little more sanity checks to make sure the stacking is correctly done
# for images

# need to save these files
valid_save_path = 'Train_TN_data/final_valid_sub_windows_and_counts'

# save the sub images
np.save(valid_save_path + "/" + "valid_sub_windows.npy", valid_im_stack)
# save the sub counts
np.save(valid_save_path + "/" + "valid_sub_counts.npy", valid_count_stack)

In [ ]:
def create_subwindows_and_counts(image, numpy_folder, stride = 8, kernel_size = 32):
    im_name = image.split(".")[0]
    im_file = im_name + '.npy'
    count_file = im_name + '_density_map.npy'
    # load the image and the count numpy files
    loaded_im_file = np.load(numpy_folder + '/' + im_file)
    loaded_count_file = np.load(numpy_folder + '/' + count_file)
        
    # create the subwindows and counts as follows
    img_height = loaded_im_file.shape[0]
    img_width = loaded_im_file.shape[1]
    
    density_sums = []
    catch_image = []
    for i in  range(0, img_height, stride):
        for j in range(0, img_width, stride):
            sub_window = loaded_im_file[i: i + kernel_size, j : j + kernel_size,:]
            density = loaded_count_file[i: i + kernel_size, j : j + kernel_size]
            dense_sum = np.sum(density)
            density_sums.append(dense_sum)
            sub_window = resize(sub_window, (32, 32,3))
            catch_image.append(sub_window)

    # save the combined subwindows and counts
    return(catch_image,density_sums, im_file)

# let's do this for a sample and then in the loop
train_files_path = "Train_TN_data/train_all_image_density_files/"

train_im_and_map_contents = os.listdir(train_files_path)

In [ ]:
train_im_names

In [ ]:
# sort these - ALWAYS sort these as the order is always messed up on HCC
train_im_and_map_contents.sort()
print(len(train_im_and_map_contents))
# get only the names of the image (npy files)
train_im_names = [item for item in train_im_and_map_contents if item.split(".")[0][-3:] != 'map']
train_im_names = [item for item in train_im_names if item != '.ipynb_checkpoints']
print(len(train_im_names))



In [ ]:
# train_im_names

In [ ]:
# create the subwindows for all train data
catch_all_image_subwindows_train = []
catch_all_dense_subwindows_train = []
catch_train_names = []
for image in train_im_names:
    train_ims, train_maps, train_names = create_subwindows_and_counts(image, train_files_path, stride = 8, kernel_size = 32)
    catch_all_image_subwindows_train.append(train_ims)
    catch_all_dense_subwindows_train.append(train_maps)
    catch_train_names.append(train_names)